In [1]:
import pandas as pd
import altair as alt
import numpy as np

alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
df = pd.read_csv('data/billionaires.csv', parse_dates=['birthDate'])

In [3]:
df

,finalWorth,personName,age,country,industries,status,gender,birthDate,gdp_country,life_expectancy_country,total_tax_rate_country,economic_class,education,continent,industry_sector,year,decade,e_class
0,211.0,Bernard Arnault & family,74.0,France,Fashion & Retail,U,Male,1949-03-05,2715.520000,82.5,60.7,1,3.0,Europe,Consumer Discretionary & Staples,1949.0,1940.0,Developed
1,80.5,Francoise Bettencourt Meyers & family,69.0,France,Fashion & Retail,U,Female,1953-07-10,2715.520000,82.5,60.7,1,NaN,Europe,Consumer Discretionary & Staples,1953.0,1950.0,Developed
2,40.1,François Pinault & family,86.0,France,Fashion & Retail,D,Male,1936-08-21,2715.520000,82.5,60.7,1,0.0,Europe,Consumer Discretionary & Staples,1936.0,1930.0,Developed
3,22.0,Emmanuel Besnier,52.0,France,Food & Beverage,D,Male,1970-09-18,2715.520000,82.5,60.7,1,NaN,Europe,Consumer Discretionary & Staples,1970.0,1970.0,Developed
4,9.8,"Jacques Saadé, Jr.",51.0,France,Logistics,Split Family Fortune,Male,1971-08-10,2715.520000,82.5,60.7,1,NaN,Europe,Service,1971.0,1970.0,Developed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2652,1.2,Zsolt Felcsuti,51.0,Hungary,Manufacturing,N,Male,1971-05-26,160.967000,75.8,37.9,2,NaN,Europe,Energy & Industrials & Materials,1971.0,1970.0,Emerging
2653,1.2,Lorinc Meszaros,57.0,Hungary,Diversified,E,Male,1966-02-25,160.967000,75.8,37.9,2,NaN,Europe,Diversified,1966.0,1960.0,Emerging
2654,1.5,Mohammed Dewji,47.0,Tanzania,Diversified,E,Male,1975-05-08,63.177068,65.0,43.8,3,NaN,Africa,Diversified,1975.0,1970.0,Developing
2655,1.5,Carlos Rodriguez-Pastor,63.0,Peru,Finance & Investments,Split Family Fortune,Male,1959-04-11,226.848000,76.5,36.8,2,4.0,Americas,Financials,1959.0,1950.0,Emerging


# Task 3

#### Task 3 - Correlation: Does the socio-economic status of a country (economic_rating, GDP, life expectancy) correlate to the country's median billionaire final net worth?

Scatterplot map faceted by the economic class of a country (there are 2 rows of 3 scatterplots side by side - developing, emerging and developed). Both graphs contain the 'Median final net worth' on the y-axis. One graph contains the country's GDP on the x-axis. The other graph contains the country's life expectancy on the x-axis.

Thoughts:
each billionaire is listed with their country. so every billionaire is also associated with their country's LE and GDP so I only need to find their median and create the graphs from there

steps/ideas:
- tooltip hovering

In [4]:
req = ['finalWorth', 'country', 'gdp_country', 'life_expectancy_country']
df[req].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2657 entries, 0 to 2656
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   finalWorth               2657 non-null   float64
 1   country                  2657 non-null   object 
 2   gdp_country              2657 non-null   float64
 3   life_expectancy_country  2657 non-null   float64
dtypes: float64(3), object(1)
memory usage: 83.2+ KB


In [5]:
#potentially delete
df[pd.isnull(df['gdp_country'])]

,finalWorth,personName,age,country,industries,status,gender,birthDate,gdp_country,life_expectancy_country,total_tax_rate_country,economic_class,education,continent,industry_sector,year,decade,e_class


In [6]:
#countries with null values in gdp_country
df[pd.isnull(df['gdp_country'])].country.unique()

array([], dtype=object)

In [7]:
#countries with null values in life_expectancy_country
df[pd.isnull(df['life_expectancy_country'])].country.unique()

array([], dtype=object)

In [8]:
#filling in the null values with the appropriate values (information obtained from Google) and manipulating dataset to fulfill needs
df['e_class'] = ''
df.loc[df['country'] == 'Hong Kong', 'gdp_country'] = 91400000000
df.loc[df['country'] == 'Hong Kong', 'life_expectancy_country'] = 85.3
df.loc[df['country'] == 'Taiwan', 'gdp_country'] = 185233000000
df.loc[df['country'] == 'Taiwan', 'life_expectancy_country'] = 81.0
df.loc[df['economic_class'] == 1, 'e_class'] = 'Developed'
df.loc[df['economic_class'] == 2, 'e_class'] = 'Emerging'
df.loc[df['economic_class'] == 3, 'e_class'] = 'Developing'


df['gdp_country'] = df['gdp_country'] / 10000000000

In [9]:
df['medianFinalWorth'] = df.groupby(['country'])['finalWorth'].transform('median')
req = ['medianFinalWorth', 'country', 'gdp_country', 'life_expectancy_country', 'economic_class', 'e_class']
task3df = df[req].drop_duplicates().reset_index().drop('index', axis=1)
task3df.head()

,medianFinalWorth,country,gdp_country,life_expectancy_country,economic_class,e_class
0,4.2,France,2.715520e-07,82.5,1,Developed
1,2.9,United States,2.142770e-06,78.5,1,Developed
2,4.0,Mexico,1.258290e-07,75.0,2,Emerging
3,2.1,India,2.611000e-07,69.4,2,Emerging
4,2.1,Spain,1.394120e-07,83.3,1,Developed


In [10]:
splot_brush = alt.selection_interval(encodings=['x', 'y'], empty=False)

In [11]:
gdp_splot = alt.Chart(task3df).mark_point().encode(
    alt.X('gdp_country', title='Country GDP (billion USD)').scale(type='log'),
    alt.Y('medianFinalWorth', title='Median Final Net Worth (million USD)'),
    color=alt.Color('e_class:N', 
             title='Economic class',
             # condition=alt.condition(splot_brush, 'e_class:N', alt.value('lightgray'))
             ),
     # color = alt.condition(splot_brush, 'e_class:N', alt.value('lightgray'), title='Economic Class'),
    tooltip=[
             alt.Tooltip('country', title='Country'),
             alt.Tooltip('gdp_country', title='Country GDP (billion USD)'),
             alt.Tooltip('medianFinalWorth', title='Median Final Net worth (million USD)'), 
             ]
).facet(
    column=alt.Column('e_class:O', sort=['Developed','Emerging','Developing']),
    # title=alt.TitleParams("Correlation between Median Billionaire Final Net Worth and Country's Socio-Economic Status",
                         # anchor='middle'),
).add_params(splot_brush
 )#.transform_filter(
#     splot_brush
#  )


life_splot = alt.Chart(task3df).mark_point().encode(
    alt.X('life_expectancy_country', title='Country Life Expectancy (Years)').scale(zero=False),
    alt.Y('medianFinalWorth', title='Median Final Net Worth (million USD)'),
    alt.Color('e_class:N'),#, legend=None),
    tooltip=[
             alt.Tooltip('country', title='Country'),
             alt.Tooltip('life_expectancy_country', title='Country Life Expectancy'),
             alt.Tooltip('medianFinalWorth', title='Median Final Net worth (million USD)'), 
             ]
).facet(
    column=alt.Column('e_class:O', sort=['Developed','Emerging','Developing']),
)

alt.vconcat(gdp_splot, life_splot).configure_header(
    title= None,
)

# gdp_splot | life_splot
# .encode(
    # color = alt.condition(splot_brush, 'e_class:N', alt.value('lightgray'))
# )
# .add_params(splot_brush)

alt.VConcatChart(...)

In [17]:
dropdown_ops = task3df['e_class'].unique().tolist()
dropdown = alt.binding_select(
    options = dropdown_ops,
    name='Economic Class: '
)

selection = alt.selection_point(
    fields=['e_class'],
    value=[{'e_class': dropdown_ops[0]}],
    bind=dropdown
)

zzz= alt.Chart(task3df).mark_point(size=40).encode(
    alt.X('gdp_country',
          title='Country GDP (billion USD)').scale(type='log'),
    alt.Y('medianFinalWorth',
          title='Median Final Net Worth (million USD)'),
    color=alt.condition(splot_brush,
                        'e_class:N',
                        alt.value('lightgray'),
                        title='Economic Class'),
    tooltip=[
             alt.Tooltip('country',
                         title='Country'),
             alt.Tooltip('gdp_country',
                         title='Country GDP (billion USD)'),
             alt.Tooltip('medianFinalWorth',
                         title='Median Final Net worth (million USD)'), 
             ]
).add_params(splot_brush)
zzz1 = life_splot = alt.Chart(task3df).mark_point(size=40).encode(
    alt.X('life_expectancy_country',
          title='Country Life Expectancy (Years)').scale(zero=False),
    alt.Y('medianFinalWorth', title='Median Final Net Worth (million USD)'),
    color=alt.condition(splot_brush,
                        'e_class:N',
                        alt.value('lightgray'),
                        title='Economic Class'),
    tooltip=[
             alt.Tooltip('country',
                         title='Country'),
             alt.Tooltip('life_expectancy_country',
                         title='Country Life Expectancy'),
             alt.Tooltip('medianFinalWorth',
                         title='Median Final Net worth (million USD)'),
             ]
).add_params(splot_brush)

task34 = (gdp_splot | life_splot.encode(
    color=alt.condition(splot_brush,
                        'e_class:N',
                        alt.value('lightgray')),
).add_params(splot_brush, selection)).properties(
    title="Median Billionaire Networth and Country's Socio-Economic Status",
).transform_filter(
    selection
)
task34

alt.HConcatChart(...)

In [13]:
splot_brush = alt.selection_interval(encodings=['x', 'y'], empty=True)
gdp_splot = alt.Chart(task3df).mark_point(size=40).encode(
    alt.X('gdp_country',
          title='Country GDP (billion USD)').scale(type='log'),
    alt.Y('medianFinalWorth',
          title='Median Final Net Worth (million USD)'),
    color=alt.condition(splot_brush,
                        'e_class:N',
                        alt.value('lightgray'),
                        title='Economic Class'),
    tooltip=[
             alt.Tooltip('country',
                         title='Country'),
             alt.Tooltip('gdp_country',
                         title='Country GDP (billion USD)'),
             alt.Tooltip('medianFinalWorth',
                         title='Median Final Net worth (million USD)'), 
             ]
).add_params(
    splot_brush
)

life_splot = alt.Chart(task3df).mark_point(size=40).encode(
    alt.X('life_expectancy_country',
          title='Country Life Expectancy (Years)').scale(zero=False),
    alt.Y('medianFinalWorth', title='Median Final Net Worth (million USD)'),
    alt.Color('e_class:N'),
    tooltip=[
             alt.Tooltip('country',
                         title='Country'),
             alt.Tooltip('life_expectancy_country',
                         title='Country Life Expectancy'),
             alt.Tooltip('medianFinalWorth',
                         title='Median Final Net worth (million USD)'),
             ]
)

task3 = (gdp_splot | life_splot.encode(
    color=alt.condition(splot_brush,
                        'e_class:N',
                        alt.value('lightgray')),
).add_params(splot_brush)).properties(
    title="Median Billionaire Networth and Country's Socio-Economic Status",
)
task3

alt.HConcatChart(...)